In [6]:
import pandas as pd
import numpy as np

In [ ]:
for year in reversed(range(1976,2020)):
    text_file = open("launch_scripts/description_features/launch_{}.sh".format(year), "w")
    text_file.write("#!/bin/bash\n")
    text_file.write("#SBATCH -n 2\n")
    text_file.write("#SBATCH -t 48:00:00\n")
    text_file.write("#SBATCH --mem=32g\n")
    text_file.write("module load python/3.6.6\n")
    text_file.write("python3 /users/jkatz/scratch/Upwork/isogriv/get_description_features.py {}".format(year))
    text_file.close()

In [ ]:
import os
for year in reversed(range(1976,2020)):
    os.system("sbatch launch_scripts/description_features/launch_{}.sh".format(year))

In [ ]:
import zipfile, os
for file in os.listdir():   # get the list of files
    if file[0:6] == "detail"
        with zipfile.ZipFile(file) as item: # treat the file as a zip
            item.extractall()  # extract it in the working directory

In [31]:
df = pd.read_csv("/Users/JasonKatz/Downloads/ptab2_decisions.csv")

/Users/JasonKatz/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,9,11,12,13,14,15,16,17,19,20,21,22,23,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
len(list(np.unique(df['subdecision_type_category'])))

51

In [ ]:
df.clip

In [39]:
df['respondent_patent_number'][0]

'8271812'

In [41]:
'8271812' in list(df['respondent_patent_number'])

True

In [57]:
for year in reversed(range(1976,2020)):
    text_file = open("python_scripts/parse_{}.py".format(year), "w")
    text_file.write("import csv\n")
    text_file.write("import pandas as pd\n")
    text_file.write("import sys\n")
    text_file.write("import json\n")
    text_file.write("patent_decisions_df = pd.read_csv('/users/jkatz/scratch/Upwork/isogriv/ptab2_decisions.csv')\n")
    text_file.write("csv.field_size_limit(sys.maxsize)\n")
    text_file.write("patent_descriptions = {}\n")
    text_file.write("patents = list(patent_decisions_df['respondent_patent_number'])\n")
    text_file.write("with open('/users/jkatz/scratch/Upwork/isogriv/detail_description_data/detail-desc-text-{}.tsv','r') as tsvin:\n".format(year))
    text_file.write("    tsvin = csv.reader(tsvin, delimiter='\t')\n")
    text_file.write("    for row in tsvin:\n")
    text_file.write("        if row[0] in patents:\n")
    text_file.write("            patent_descriptions[row[0]] = row[1]\n")
    text_file.write("with open('patent_descriptions/descriptions_{}.json', 'w') as f:\n".format(year))
    text_file.write("    json.dump(patent_descriptions, f)\n")
    text_file.close()

SyntaxError: EOL while scanning string literal (<ipython-input-57-258fb182bca1>, line 4)

In [51]:
import csv
import sys
impor

patent_decisions_df = pd.read_csv("/Users/JasonKatz/Downloads/ptab2_decisions.csv")
csv.field_size_limit(sys.maxsize)
patent_descriptions = {}
patents = list(patent_decisions_df['respondent_patent_number'])
with open('/Users/JasonKatz/Downloads/detail_desc_text_2019.tsv','r') as tsvin:
    tsvin = csv.reader(tsvin, delimiter='\t')

    count = 0
    for row in tsvin:
        if row[0] in patents:
            patent_descriptions[row[0]] = row[1]
with open('patent_descriptions/descriptions_{}.json', 'w') as f:
    json.dump(data, f)

In [10]:
"{0:01}".format(int('099')+1)

'100'

In [34]:
import os
numbers = ["{0:03}".format(i) for i in reversed(range(99,200))]

In [35]:
for number in numbers:
    file_name = "detail_description_{}.tsv".format(number)
    new_num = "{0:01}".format(int(number)+1)
    new_file_name = "detail_description_{}.tsv".format(new_num)
#     print("{} --> {}".format(file_name, new_file_name))
    os.system("mv {} {}".format(file_name, new_file_name))

detail_description_199.tsv --> detail_description_200.tsv
detail_description_198.tsv --> detail_description_199.tsv
detail_description_197.tsv --> detail_description_198.tsv
detail_description_196.tsv --> detail_description_197.tsv
detail_description_195.tsv --> detail_description_196.tsv
detail_description_194.tsv --> detail_description_195.tsv
detail_description_193.tsv --> detail_description_194.tsv
detail_description_192.tsv --> detail_description_193.tsv
detail_description_191.tsv --> detail_description_192.tsv
detail_description_190.tsv --> detail_description_191.tsv
detail_description_189.tsv --> detail_description_190.tsv
detail_description_188.tsv --> detail_description_189.tsv
detail_description_187.tsv --> detail_description_188.tsv
detail_description_186.tsv --> detail_description_187.tsv
detail_description_185.tsv --> detail_description_186.tsv
detail_description_184.tsv --> detail_description_185.tsv
detail_description_183.tsv --> detail_description_184.tsv
detail_descrip

In [ ]:
import pandas as pd
import os
import json
from joblib import dump, load

In [ ]:
all_description_features = []
for file_name in os.listdir("probability_data/"):
    with open("probability_data/{}".format(file_name)) as f:
        description_features = json.load(f)
    for key,val in description_features.items():
        all_description_features.append([key] + val)
        
all_description_features_df = pd.DataFrame(all_description_features, columns=['patent_id', 'description_word_count', 
                                                                              'fk_score', 'fig_counts'])
all_description_features_df['patent_id'] = all_description_features_df['patent_id'].astype(str)

In [ ]:
days_between_features = pd.read_csv("patent_days_between.csv", index_col=0).rename(columns={'id':'patent_id'})
days_between_features['patent_id'] = days_between_features['patent_id'].astype(str)

In [ ]:
patent_data_preds = all_description_features_df.merge(days_between_features, on="patent_id", how="inner")

model = load("patent_GBC.joblib")

patent_data_preds['good_claim_probability'] = model.predict_proba(patent_data_preds.iloc[:,1:])[:,1]

In [ ]:
patent_data_preds.to_csv("patent_data_preds.csv")

In [ ]:
import os
os.system("cp detail_description_data/detail-desc-text-2019.tsv all_detail_descriptions.tsv")
for year in reversed(range(1976,2019)):
    os.system("tail --lines=+2 detail_description_data/detail-desc-text-{}.tsv >> all_detail_descriptions.tsv".format(year))

In [ ]:
all_patent_descriptions = {}
for year in range(1976,2020):
    with open("patent_descriptions/descriptions_{}.json".format(year)) as f:
        patent_descriptions = json.load(f)
    for key,val in patent_descriptions.items():
        all_patent_descriptions[key] = val
        
all_patent_descriptions_df = pd.DataFrame(list(zip(list(all_patent_descriptions.keys()),list(all_patent_descriptions.values()))), columns=["patent_id", "detailed_description"])